# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


In [1]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host:str="DBMS 서버 ip",
                                     port:int=port번호,
                                     user:str="계정명",
                                     password:str="비밀번호",
                                     db:str="연결할데이터베이스이름")
    ```
    - port 번호 기본값: 3306
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체. 즉, 입출력 작업을 해주는 애
    ```python
        cursor = connection.cursor()
    ```
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
3. Cusror를 이용해 SQL문 실행(DB Server로 전송)
    ```python
        cursor.execute("sql문")
    ```
4. Select 결과 조회
    - select 문을 실행한 경우 cursor를 의 fetch 메소드들을 이용해 select 결과를 조회한다.
    ```python
    result = cursor.fetchall()
    ```
5. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [5]:
create_sql = """
create table customer(
  id  int  auto_increment  primary key,
  name  varchar(20) not null,
  email  varchar(50) not null unique, 
  tall   double,
  birthday  date,
  created_at  datetime  not null
)
"""
# sql 문 마지막에 `;` 은 붙이지 않는다.

In [6]:
import pymysql # pymysql import하는 것 까먹지 말기

try:
    conn = None # connection을 저장할 변수
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user='nggw519',        # username: str 대문자/소문자 주의!!
        password="1111",    # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()
        
    # 3. SQL 문 전송
    cursor.execute("drop table if exists customer") # 1 만약 customer가 있으면 먼저 지우고 만든다!
    cursor.execute(create_sql) 
    
finally:
    # 4. 연결닫기(끊기)
    # 만약 connection을 하는 과정에서 에러가 생겼다면, 애초에 conn이 발생하지않아 close 할 필요도 없다.
    # 이경우 exception이 생겨 finally구문에서는 if조건절이 None(false)이므로 close 실행 안함
    # but 정상적으로 작동해서 conn이 생긴다면 조건절이 True가 되어 close한다
    if conn: # conn != None  -> 최초에 conn에 None을 담아두었다. 
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()   # 4-2. connection 연결 닫기

## 파이썬 타입과 연결된 sql 데이터타입 
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 실수(float, double)
- decimal.Decimal - 실수(decimal)
- datetime.date - date
- datetime.time - time
- datetime.datetime - datetime, timestamp

> ### datetime 모듈
> - 파이썬에서 날짜, 시간을 다루는 모듈
> - 날짜 type(class): date
> - 시간 type: time
> - 날짜시간 type: datetime

In [12]:
# datetime 모듈 쓰는법
# datetime은 파이썬에서 제공하는 내장된 표준모듈

from datetime import date, time, datetime

#실행시점
a = datetime.now()
print(a)
print(a.year, a.month ,a.day, a.hour, a.minute, a.second)
b = date.today() # 날짜만!!
print(b)

# 특정 시점의 일시
c = date(2000, 10, 2)
print(c)
d = datetime(1990, 2, 7, 10, 22, 53)
print(d)
e = time(17, 22, 33)
print(e)

2025-04-08 10:12:12.440025
2025 4 8 10 12 12
2025-04-08
2000-10-02
1990-02-07 10:22:53
17:22:33


## DML

### insert


In [13]:
sql = "insert into customer (name, email, tall, birthday, created_at) values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"
# id는 auto_increment라서 생략함
# values 넣을때 작은따옴표!!

In [14]:
# with 문을 이용해 connection, cursor 생성: with block을 빠져 나올 때 자동으로 close() 처리한다.
#  DML(insert/update/delete) 처리 후 commit을 실행해야 영구적으로 적용된다.
with pymysql.connect(host="127.0.0.1", port=3306, user="nggw519", password="1111", db="mydb") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql) # execute는 무조건 반환값이 있다. 반환값: 괄호안에 넣어준 쿼리 내에서 처리된 문장 행수(insert/delete/update된행수, select 조회행수)
        print("처리 행수:", result)
        conn.commit()
# execute를 하면 메모리에(DBMS) sql문이 임시적으로 저장된다. 이후 commit을 해줘야 db내에 영구적으로 저장이 된다.
# commit 전에는 아직 db에 저장된게 아니라서, db를 확인해보면 DML이 처리되기 전이다.
# commit되기 전에는 rollback 가능

처리 행수: 1


### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   동일한 쿼리문을 값을 바꿔가면서 여러번 실행할 때 유용하다.


In [10]:
import pymysql

# insert할 값 입력받기
name = input("이름:")
email = input("이메일주소:")
tall = float(input("키:"))

insert_sql = "insert into customer (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())"

with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, tall, "2010-01-01"]) # 리스트나 튜플로 묶어주면된다.
        conn.commit()
        print("처리 행수:", result)

이름: 김철수
이메일주소: ads@ds.d
키: 179


처리 행수: 1


#### Parameterized Query를 이용해 여러개 행 insert

- executemany() 사용
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [11]:
from datetime import datetime, date
datas = [
    ["김인영", "abc2@a.com", 165, date(2005, 1, 12), datetime.now()],
    ["오수철", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["최유명", "ghi2@a.com", 183, date(1978, 10, 28), datetime.now()],
    ["김명수", "jkl@abc.com", 177, date(2000, 2, 12), datetime.now()],
    ["이지영", "mno@abc.com", 163, date(1995, 4, 21), datetime.now()],
    ["박명수", "pqr@abc.com", 185, date(2002, 7, 5), datetime.now()],
]

In [12]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas) # execute는 하나씩 처리해주지만 executemany는 datas에 리스트로 된것을 넣어주면 for문으로 돌려주는 것.
        conn.commit()

print("insert된 총 행수:", cnt)


# 아래 두개 코딩은 같은것
# 1. cursor.executemany(insert_sql, datas)
# 2. for data in datas
#        cursor.executemany(insert_sql, datas)

insert된 총 행수: 6


### update/delete

-   코딩 절차는 insert 와 동일


In [22]:
update_sql = "update customer set tall=%s where id=%s"
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: ")) # 6번 오수철씨 키 바꿔보기

with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 키:  177.2
변경할 고객 ID:  6


처리 행수:  1


In [24]:
update_sql = "update customer set email=%s, tall=%s where id=%s"
email = input("변경할 Email주소: ")
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: ")) # 마찬가지 6번 바꿔보기

with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [email, tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 Email주소:  such@sad.com
변경할 키:  174
변경할 고객 ID:  6


처리 행수:  1


In [25]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:")) #180 이상만 삭제하기
with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, [tall])
        print("처리 행수: ", result)
        conn.commit()

삭제기준 키: 180


처리 행수:  4


## select (DQL - Data Query Language)

-   조회결과 조회
    -   `cursor.execute("select문")` 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
    -   select 실행된 결과가 메모리에 저장되고, fetch를 이용하여 메모리에서 결과를 가져온다.
    -   위에서 배운 delete등의 구문과 달리 select은 execute이 처리한 행수가 중요하지 않다.
    -   따라서 fetchall로 조회한 행을 보야야한다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환. 
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [28]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [29]:
resultset
# tuple(개별행 - tuple(컬럼값들 ))

((3, '양금명', 165.88),
 (4, '양은명', 179.6),
 (5, '김인영', 165.0),
 (6, '오수철', 174.0),
 (8, '김명수', 177.0),
 (9, '이지영', 163.0))

In [30]:
resultset[0][1]

'양금명'

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
    - 컴럼명도 같이 보고싶을 때 쓴다.
- key: 컬럼명, value: 컬럼값

In [32]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [33]:
resultset
# list[dictionary{key:컬럼명, value:컬럼값}]

[{'id': 3, 'name': '양금명', 'tall': 165.88},
 {'id': 4, 'name': '양은명', 'tall': 179.6},
 {'id': 5, 'name': '김인영', 'tall': 165.0},
 {'id': 6, 'name': '오수철', 'tall': 174.0},
 {'id': 8, 'name': '김명수', 'tall': 177.0},
 {'id': 9, 'name': '이지영', 'tall': 163.0}]

In [34]:
resultset[1]['name']

'양은명'

### fetchone()


In [13]:
sql = "select * from customer where id = %s" 
with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [5])  #(2, ) []괄호 안은, PK의 값을 찾아간다.
        print("조회행수:", result)
        resultset = cursor.fetchone()
        # resultset = cursor.fetchall()

조회행수: 1


In [14]:
if resultset:  # 조회결과가 없으면 None
    print(resultset)
else:
    print("조회결과가 없음.")
  

(5, '김명수', 'jkl@abc.com', 177.0, datetime.date(2000, 2, 12), datetime.datetime(2025, 4, 8, 11, 55, 17))


In [15]:
  # fetchall을 쓰면 튜플로 묶어서 주기 때문에, 만약 이메일만 조회하고싶으면 인덱스를 따로 지정해야한다.
resultset[2]

'jkl@abc.com'

### fetchmany()


In [16]:
sql = "select id, name, birthday from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)
        resultset4 = cursor.fetchmany(size=2)

조회행수: 7


In [21]:
print(resultset1)
print(resultset2)
print(resultset3)
print(resultset4)
print(resultset1[1][1])

((1, '김철수', datetime.date(2010, 1, 1)), (2, '김인영', datetime.date(2005, 1, 12)))
((3, '오수철', datetime.date(1995, 12, 20)), (4, '최유명', datetime.date(1978, 10, 28)))
((5, '김명수', datetime.date(2000, 2, 12)), (6, '이지영', datetime.date(1995, 4, 21)))
((7, '박명수', datetime.date(2002, 7, 5)),)
김인영


### select문을 실행한 cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.
-   cursor를 돌려서 한 행씩 처리하고 싶을때 사용한다.


In [63]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        for id, name, email, tall, birthday, created_at in cursor: # cursor는 튜플이므로 for문의 변수를 왼쪽과같이 적어줄 수 있다.
            print(id, name, email, tall, birthday, created_at, sep=" , ")

조회행수: 6
3 , 양금명 , gold@ps.com , 165.88 , 2010-01-01 , 2025-04-08 10:32:57
4 , 양은명 , silver@ps.com , 179.6 , 2010-01-01 , 2025-04-08 10:33:38
5 , 김인영 , abc2@a.com , 165.0 , 2005-01-12 , 2025-04-08 10:37:10
6 , 오수철 , such@sad.com , 174.0 , 1995-12-20 , 2025-04-08 10:37:10
8 , 김명수 , jkl@abc.com , 177.0 , 2000-02-12 , 2025-04-08 10:37:10
9 , 이지영 , mno@abc.com , 163.0 , 1995-04-21 , 2025-04-08 10:37:10


In [82]:
#####################
# SQL 실행코드를 함수화
#####################

def select_customer_by_id(cust_id:int) -> tuple|None: #|(세로줄)은 or이라는 뜻
    """
    고객 id로 정보를 DB에서 조회해서 반환하는 함수.
    Args:pass
    Returns
        tuple: 조회결과 있으면 튜플로 리턴
        None: 조회결과 없으면 None으로 리턴
    Raises:pass
    """
    sql = "select * from customer where id = %s" 
    with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, [cust_id])
            print("조회행수:", result)
            return cursor.fetchone()



In [73]:
result = select_customer_by_id(5)
print(result)

조회행수: 1
(5, '김인영', 'abc2@a.com', 165.0, datetime.date(2005, 1, 12), datetime.datetime(2025, 4, 8, 10, 37, 10))


In [74]:
result = select_customer_by_id(1) #없는 사람이면 None 반환
print(result)

조회행수: 0
None


In [76]:
# 문장이 너무 길때 실제론 1줄인데 나눠서 보이게하느 방법
# 1.\ 사용
"sssssssssssssssssssssssssssssssssssssssssssss\
ssssssssssssssssssssssssssssssssssssssssssssssssssss"

 # 2. 괄호 사용(쉼표 없음 주의)
(
    "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
    "bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"
    "ccccccccccccccccccccccccccccccccccccccccccccccccc"
)

'sssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss'

In [83]:
def update_customer(cust_id:int, name:str, email:str, tall:float, birthday:date|str) -> int:

    sql = ("update customer "
           "set name=%s, email=%s, tall=%s, birthday=%s " # 공백 남기기 중요! 안남기면 붙는다.
           "where id=%s")
    with pymysql.connect(host="127.0.0.1", port=3306, user='nggw519', password='1111', db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, (name, email, tall, birthday, cust_id))
            return result

In [84]:
update_customer(5,"김인영", "new@ds.com", 165, '2005-03-02')
# 바꾸지 않을건 그냥 원래 값 넣어 주면된다.

1